# Setup

In [ ]:
#@title Dependencies and Imports
%load_ext autoreload
%autoreload 2
%pip install -q lightning click transformers goatools toml wget fastobo torch_xla

from google.colab import userdata, drive
import os
from pathlib import Path


In [ ]:
#@title Environment Variables
github_pat = userdata.get("GITHUB_PAT")
wandb_key = userdata.get("WANDB_API_KEY")
os.environ["WANDB_API_KEY"] = wandb_key
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Clone and cd
if os.getcwd() != "/content/contempro/work":
  if not Path("/content/contempro").exists():
    !git clone https://{github_pat}@github.com/boun-tabi-lifelu/contempro.git
  %cd /content/contempro/work

/content/contempro/work


# Data Fetching

In [ ]:
#@title Setup data
!mkdir -p datasets
!cp /content/drive/MyDrive/research/contempro/work-pfresgo-data.zip ./datasets
!cd datasets && unzip -o work-pfresgo-data.zip

Archive:  work-pfresgo-data.zip
  inflating: __MACOSX/._pfresgo      
  inflating: pfresgo/.DS_Store       
  inflating: __MACOSX/pfresgo/._.DS_Store  
  inflating: pfresgo/annot.tsv       
  inflating: __MACOSX/pfresgo/._annot.tsv  
  inflating: pfresgo/nrPDB-GO_2019.06.18_test.csv  
  inflating: __MACOSX/pfresgo/._nrPDB-GO_2019.06.18_test.csv  
  inflating: pfresgo/train.txt       
  inflating: __MACOSX/pfresgo/._train.txt  
  inflating: pfresgo/go.obo          
  inflating: __MACOSX/pfresgo/._go.obo  
  inflating: pfresgo/ontology.embeddings.npy  
  inflating: __MACOSX/pfresgo/._ontology.embeddings.npy  
  inflating: pfresgo/valid.txt       
  inflating: __MACOSX/pfresgo/._valid.txt  
  inflating: pfresgo/nrPDB-GO_2019.06.18_sequences.fasta  
  inflating: __MACOSX/pfresgo/._nrPDB-GO_2019.06.18_sequences.fasta  
  inflating: pfresgo/test.txt        
  inflating: __MACOSX/pfresgo/._test.txt  


In [ ]:
!cp /content/drive/MyDrive/research/per_residue_embeddings.h5 ./datasets/pfresgo

# Test DataModule

In [ ]:
%load_ext autoreload
%autoreload 2
from data.datamodule import PFresGODataModule

dm = PFresGODataModule(
    data_dir="./datasets/pfresgo",
    ontology="molecular_function",
    num_workers=10
)
dm.setup()
dl = dm.train_dataloader()
next(iter(dl))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/mmtf/p/research/contempro/work/.pixi/envs/default/lib/python3.13/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/Users/mmtf/p/research/contempro/work/.pixi/envs/default/lib/python3.13/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ~~~~~~~~~~~~~~~^^^^^^
  File "/Users/mmtf/p/research/contempro/work/data/datamodule.py", line 236, in collate_fn
    seq_emb_dim = batch[0].residue_embeddings.shape[1]
                  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
IndexError: tuple index out of range


# Training

In [ ]:
%run -i train.py

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mmtftr (mmtf). Use `wandb login --relogin` to force relogin


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for p

True
